In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Define paths for your train, validation, and test data
train_dir = "output_folder/train"
val_dir = "output_folder/val"
test_dir = "output_folder/test"

In [30]:
# Define data generators with data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(   
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 5450 images belonging to 102 classes.
Found 1830 images belonging to 102 classes.
Found 1864 images belonging to 102 classes.


In [31]:
# Define the number of samples in the training dataset
num_samples = len(train_generator)

# Initialize empty lists to store features (x_train) and labels (y_train)
x_train = []
y_train = []
x_val = []
y_val = []

# Loop through the batches generated by train_generator
for i in range(num_samples):
    x_batch, y_batch = train_generator[i]  # Get a batch of data
    x_train.append(x_batch)
    y_train.append(y_batch)
print(len(validation_generator))
for i in range(0,10):
    x_batch, y_batch = validation_generator[i]  # Get a batch of data
    x_val.append(x_batch)
    x_val.append(y_batch)

# Concatenate the lists to create NumPy arrays
x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)
# x_val = np.concatenate(x_val)
# y_val = np.concatenate(y_val)

58


In [32]:
len(x_train)

5450

In [33]:
# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom classification layers on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(102, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pre-trained model (optional)
for layer in base_model.layers:
    layer.trainable = False

In [35]:
# Compile the model
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on your data
model.fit(x_train, y_train, epochs=10)


Epoch 1/10
171/171 [==============================] - 11s 61ms/step - loss: 3.7160 - accuracy: 0.2222
Epoch 2/10
171/171 [==============================] - 10s 56ms/step - loss: 2.9572 - accuracy: 0.3495
Epoch 3/10
171/171 [==============================] - 10s 58ms/step - loss: 2.6030 - accuracy: 0.4046
Epoch 4/10
171/171 [==============================] - 10s 58ms/step - loss: 2.3925 - accuracy: 0.4393
Epoch 5/10
171/171 [==============================] - 9s 54ms/step - loss: 2.2390 - accuracy: 0.4717
Epoch 6/10
171/171 [==============================] - 10s 61ms/step - loss: 2.1178 - accuracy: 0.4965
Epoch 7/10
171/171 [==============================] - 10s 58ms/step - loss: 2.0113 - accuracy: 0.5088
Epoch 8/10
171/171 [==============================] - 10s 57ms/step - loss: 1.9286 - accuracy: 0.5288
Epoch 9/10
171/171 [==============================] - 10s 56ms/step - loss: 1.8272 - accuracy: 0.5534
Epoch 10/10
171/171 [==============================] - 10s 57ms/step - loss: 1.7533